In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd 'drive/My Drive/Colab Notebooks/Text relevance'

/content/drive/My Drive/Colab Notebooks/Text relevance


In [3]:
 %run prepare_some_data.ipynb

     |████████████████████████████████| 7.1MB 5.0MB/s 
     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 378kB 13.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=867689 sha256=3a69e843b4455eca9e431ac15564cc1c9c80074eb49b317cd1b759cefb421881
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
хороший
     |████████████████████████████████| 1.4MB 4.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=d442c8068dd2f8591baa51e23f6f729ee6b4ef8d9323d85c397617bcf9d8e0d8
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
queries=dict()
with open('queries.fixed.txt',encoding='utf-8-sig') as f:
    for line in f:
        query_id,words=line.split('\t')
        queries[int(query_id)]=process_string(words).strip().replace('.',' ')

In [0]:
candidates,reverse_candidates=generate_candidates()

In [0]:
def load_bm():
    with open('bm25res.txt','r') as f:
        ans=dict()
        for line in f:
            query_id,s=line.strip().split('\t')
            query_id=int(query_id)

            score_bm=defaultdict(float)
            for score in s.split():
                score,doc_id=score.split(',')
                score_bm[int(doc_id)]=float(score)
            ans[query_id]=score_bm
        return ans

In [7]:
!pip install transformers

     |████████████████████████████████| 665kB 4.9MB/s 
     |████████████████████████████████| 1.1MB 17.7MB/s 
     |████████████████████████████████| 3.8MB 39.7MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4e8c97c8255a35c0edcc40a5e324c056d5e516eed632d456d9a59d856a337507
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch
import timeit
from transformers import *
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
device = 'cuda'
model = model.to(device)
model.eval()
None

In [11]:
t1=tokenizer.encode("мультивиза в израиль какие страны можно посетить",add_special_tokens=False)
t2=tokenizer.encode("Хороший плохой дядя вася",add_special_tokens=False)
batch = tokenizer.batch_encode_plus([t1,t2], add_special_tokens=True, pad_to_max_length = True)['input_ids']
batch = torch.tensor(batch)
batch.size()

torch.Size([2, 11])

In [0]:
coss = torch.nn.CosineSimilarity(dim = 1)

In [13]:
a=torch.tensor([[1,2],[3,4],[5,6]]).float()
b=torch.tensor([[5,6]]).float()

print(a.size())
print(b.size())

coss(a,b)

torch.Size([3, 2])
torch.Size([1, 2])


tensor([0.9734, 0.9987, 1.0000])

In [14]:
get_query[21]

'что означает знак бесконечность '

In [15]:
query_embs=dict()
coss = torch.nn.CosineSimilarity(dim = 0)

for query_id in range(1,400):
    tmp=tokenizer.batch_encode_plus([get_query[query_id]],
                                               add_special_tokens=True, pad_to_max_length = False)
    batch=torch.tensor(tmp['input_ids'])
    batch=batch.to('cuda')
    with torch.no_grad():
        embs = torch.tensor(model(batch)[0][0])
    query_embs[query_id]=torch.mean(embs, dim = 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [0]:
def gen(batch,batch_size=32):
    for i in range(batch.size(0)//batch_size+int(batch.size(0)%batch_size!=0)):
        yield batch[i*batch_size:(i+1)*batch_size]

def divide_text(text,n):
    t=text.split()
    return [' '.join(t[i*n:(i+1)*n]) for i in range(len(t)//n+int(len(t)%n!=0))]

def addset(s,text):
    if text in s:
        return False
    else:
        s.add(text)
        return True

In [33]:
file_names=['new_bert_score.txt','bert_score1.txt','bert_score2.txt','bert_score3.txt']
i1=0
i2=10000
for n in range(0,4):
    i1=n*10000
    i2=(n+1)*10000


    with open(file_names[n],'w') as out:
        with open('bert_docs.txt','r', errors = 'ignore') as f:
            beg=time()
            for i,line in enumerate(f):
                if i>=i2 or i<i1:
                    continue
                doc_id,desc,author,keywords,title,word=line.strip('\n').split('\t')

                texts=list(filter(lambda x:x and any(map(lambda x:x!=' ',x)),
                                  title.split('.')+author.split('.')+
                                  desc.split('.')+keywords.split('.')+word.split('.')
                                 ))


                old_texts=texts
                
                for query_id in reverse_candidates[int(doc_id)]:
                    
                    texts=old_texts
                    query = queries[query_id]                    

                    texts=[new_text for text in texts for new_text in divide_text(text,3*len(query.split()))]
                    
                    texts = texts[:400]

                    texts=list(map(lambda text:' '.join(text.split()),texts))
                    
                    s=set()

                    texts=list(filter(lambda text:addset(s,text),texts))

                                                                           
                    

                    out.write(doc_id+'\t'+query)
                    if texts:
                        tmp=tokenizer.batch_encode_plus([*texts],
                                            add_special_tokens=True, pad_to_max_length = True)
                        batch=torch.tensor(tmp['input_ids'])
                        batch=batch.to('cuda')



                        all_cosss=torch.empty(0,device='cuda')

                        batch_size = (305*35//batch.size(1))
                        for little_batch in gen(batch,batch_size):
                            with torch.no_grad():
                                embs = torch.tensor(model(little_batch)[0])
                                embs=torch.sum(embs, dim = 1)/(little_batch!=0).sum(dim=1)[...,None]
                                coss=torch.nn.CosineSimilarity(dim = 1)
                                cosss=coss(embs,query_embs[query_id].unsqueeze(0))
                                all_cosss=torch.cat([all_cosss,cosss])
                                del embs
                        tmp=torch.argsort(all_cosss,descending=True)


                        for ind in tmp:
                            out.write('\t'+str(all_cosss[ind].item())+','+texts[ind])
                            
                    out.write('\n')

                if i%10==0:
                    print('i=',i)          
            print((time()-beg)/(i+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


i= 0
i= 10
i= 20
i= 30
i= 40


KeyboardInterrupt: ignored

In [20]:
get_query_id['как приготовить вкусный соус к мантам ']

60

In [0]:
file_names=['new_bert_score.txt','bert_score1.txt','bert_score2.txt','bert_score3.txt']
with open('full_bert_score.txt', 'w') as out:
    for fname in file_names:
      with open(fname,'r') as f:
          for line in f:
              out.write(line)

In [0]:
new_passage_score=defaultdict(lambda :defaultdict(list))
with open('full_bert_score.txt','r') as f:
    for line in f:
        a=line.strip('\n').split('\t')
        doc_id=int(a[0])
        query_id=get_query_id[a[1]]
        if len(a)>2:
            num = min(len(a[2:]), 10)
            new_passage_score[query_id][doc_id] = []
            for i in range(num):
                el = a[i + 2].split(',')
                new_passage_score[query_id][doc_id].append(float(el[0])) #(float(a[0],a[1])) таким образом увидишь предложение#на котором лучший score
            new_passage_score[query_id][doc_id] = sorted(new_passage_score[query_id][doc_id], reverse = True)
        else:
            new_passage_score[query_id][doc_id]=[0]

In [27]:
for qid in range(1,400):
    for did in new_passage_score[qid]:
      new_passage_score[qid][did] = (sum(new_passage_score[qid][did]) - 6)*10
sorted(new_passage_score[1].items(),key=lambda x:x[1], reverse = True)

[(74, 23.575311303138733),
 (51, 14.924426078796387),
 (78, 14.832701683044434),
 (33, 14.520832300186157),
 (83, 13.841354846954346),
 (75, 12.95711636543274),
 (70, 12.8143709897995),
 (40, 12.296384572982788),
 (85, 12.055616974830627),
 (82, 12.043102383613586),
 (95, 11.960473656654358),
 (46, 11.84893012046814),
 (87, 11.711156964302063),
 (10, 11.55693233013153),
 (88, 11.486310958862305),
 (93, 11.380895972251892),
 (41, 11.176677942276001),
 (39, 11.161603331565857),
 (35, 10.996569395065308),
 (48, 10.916077494621277),
 (60, 10.802944898605347),
 (37, 10.7686847448349),
 (5, 10.74406385421753),
 (18, 10.575344562530518),
 (26, 10.49280047416687),
 (6, 10.433235764503479),
 (29, 10.432743430137634),
 (96, 10.420588850975037),
 (32, 10.415822863578796),
 (15, 10.316182971000671),
 (12, 10.19524335861206),
 (63, 10.18180787563324),
 (71, 10.020356178283691),
 (1, 10.001189112663269),
 (24, 9.902076721191406),
 (73, 9.890551567077637),
 (4, 9.841912984848022),
 (94, 9.79569375514

In [0]:
ans='QueryId,DocumentId\n'
for query_id in range(1,400):
    tmp=sorted(new_passage_score[query_id].items(),key=lambda x:x[1], reverse = True)[:10]
    ans=reduce(lambda ans,score:ans+str(query_id)+','+str(score[0])+'\n',tmp,ans)
    
with open('bertans.txt','w') as f:
    f.write(ans)

In [0]:
bm_score=load_bm()
passage_score=new_passage_score

ans='QueryId,DocumentId\n'

alpha1=1
alpha2=100

for query_id in range(1,len(get_query)+1):
    scores=dict()
    for doc_id in candidates[query_id]:
        scores[doc_id]=(bm_score[query_id][doc_id]*alpha1+
                        passage_score[query_id][doc_id]*alpha2)
    scores=sorted(scores.items(),key=lambda x:x[1], reverse = True)[:10]
    ans=reduce(lambda ans,score:ans+str(query_id)+','+str(score[0])+'\n',scores,ans)

In [0]:
i = 11
with open(str(i) + 'combans.txt','w') as f:
    f.write(ans)